<a href="https://www.kaggle.com/code/sonawanelalitsunil/kaggle-llm-science-exam?scriptVersionId=210057461" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv


In [2]:
import pandas as pd
from string import Template
from pathlib import Path

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [5]:
test = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
train.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [6]:
train.isnull().sum()

id        0
prompt    0
A         0
B         0
C         0
D         0
E         0
answer    0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      200 non-null    int64 
 1   prompt  200 non-null    object
 2   A       200 non-null    object
 3   B       200 non-null    object
 4   C       200 non-null    object
 5   D       200 non-null    object
 6   E       200 non-null    object
 7   answer  200 non-null    object
dtypes: int64(1), object(7)
memory usage: 12.6+ KB


In [8]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the model name
llm = "t5-small"  # Replace with your desired model name

# Determine the device to use (GPU or CPU)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(llm).to(device)
tokenizer = T5Tokenizer.from_pretrained(llm)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = T5ForConditionalGeneration.from_pretrained(llm).to(device)
tokenizer = T5Tokenizer.from_pretrained(llm)

In [10]:
llm = '/kaggle/input/flan-t5/pytorch/base/'

In [11]:
from string import Template

In [12]:
preamble = \
    'Answer the following question by outputting the letters A, B, C, D, and E '\
    'in order of the most likely to be correct to the to least likely to be correct.'

template = Template('$preamble\n\n$prompt\n\nA) $a\nB) $b\nC) $c\nD) $d\nE) $e')

In [13]:
def format_input(data, index):
    # Extract values from the DataFrame
    prompt = data.loc[index, 'prompt']      # Make sure 'prompt' is a valid column
    a = data.loc[index, 'A']                # Adjust column names as necessary
    b = data.loc[index, 'B']
    c = data.loc[index, 'C']
    d = data.loc[index, 'D']
    e = data.loc[index, 'E']
    
    # Create the input text using a template
    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d, e=e
    )
    return input_text


In [14]:
print(format_input(test, 0))

Answer the following question by outputting the letters A, B, C, D, and E in order of the most likely to be correct to the to least likely to be correct.

Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

A) MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
B) MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
C) MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
D) MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy cl

In [15]:
# Step 3: Generate inputs and outputs
inputs = tokenizer(format_input(test, 0), return_tensors="pt").to(device)
outputs = model.generate(**inputs)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Step 4: Print the generated answer
print(answer)

['in galaxy clusters by demonstrating that the mass is in the form of neutrinos']


# Post-processing

In [16]:
def post_process(predictions):
    valid = set(['A', 'B', 'C', 'D', 'E'])
    # If there are no valid choices, return something and hope for partial credit
    if set(predictions).isdisjoint(valid):
        final_pred = 'A B C D E'
    else:
        final_pred = []
        for prediction in predictions:
            if prediction in valid:
                final_pred += prediction
        # add remaining letters
        to_add = valid - set(final_pred)
        final_pred.extend(list(to_add))
        # put in space-delimited format
        final_pred = ' '.join(final_pred)
        
    return final_pred

In [17]:
print(test.columns)

Index(['id', 'prompt', 'A', 'B', 'C', 'D', 'E'], dtype='object')


In [18]:
from pathlib import Path

# Ensure the correct file path
data_path = Path('/kaggle/input/kaggle-llm-science-exam')

# Read the submission file
submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='id')

# Assuming you have `test` DataFrame and `tokenizer`, `model`, `device` properly defined
for idx in test.index:
    # Format the input and tokenize
    inputs = tokenizer(format_input(test, idx), return_tensors="pt").to(device)
    
    # Generate output using the model
    outputs = model.generate(**inputs)
    
    # Decode the generated output
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Post-process the answer before storing in the submission
    submission.loc[idx, 'prediction'] = post_process(answer)

# Save the submission file
submission.to_csv("submission.csv")

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


In [19]:
submission.head()

,prediction
id,
0,A B C D E
1,A B C D E
2,A B C D E
3,A B C D E
4,A B C D E


In [20]:
submission.to_csv('submission.csv')